# Insurance Claim Analysis Scenario with Semantic Kernel and Azure OpenAI Assistants API

## Context

In May 2024, Hurricane Alicia strikes the Gulf Coast near Houston, Texas, causing extensive property damage. Fabrikam Insurance, a leading commercial insurer, now faces numerous complex claims. One such claim is from Contoso Electronics Inc. for their retail location #TX-230 in Houston.

The claim involves severe roof damage, flooding of inventory storage, compromised electrical systems, and potential business interruption. The insurer’s underwriting and claims team uses Semantic Kernel agents to orchestrate a multi-step analysis:

- **Review** the claim details from the adjuster’s report.
- **Verify** policy coverage, deductibles, and limits.
- **Assess** the severity of damage, detect any fraud indicators, and recommend next steps.

## Multi-Agent Process Using Semantic Kernel's Agent Framework

We orchestrate a multi-agent process:

1. **Claims Analyst**: Reviews claim documents, identifies key details, and notes any missing information.
2. **Policy Verifier**: Checks coverage limits, applies correct deductibles, and identifies endorsements or exclusions.
3. **Risk Assessor**: Evaluates the severity of the incident against the weather data, assesses mitigation efforts, and provides forward-looking recommendations.

The `AgentGroupChat` coordinates these agents until a final recommendation is formed.

In [ ]:
!pip install semantic-kernel==1.17.0 python-dotenv aiofiles nest_asyncio azure-search-documents pyperclip


## Imports & Configuration

In [15]:
import asyncio
import os
from pathlib import Path
from datetime import datetime, timedelta
from dotenv import load_dotenv
import logging

from semantic_kernel.kernel import Kernel
from semantic_kernel.agents.open_ai.azure_assistant_agent import AzureAssistantAgent
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy import KernelFunctionSelectionStrategy
from semantic_kernel.agents.strategies.termination.kernel_function_termination_strategy import KernelFunctionTerminationStrategy
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt

import nest_asyncio
nest_asyncio.apply()

load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-10-01-preview")

if not all([AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_DEPLOYMENT]):
    raise ValueError("Missing Azure OpenAI configuration.")

logging.getLogger().setLevel(logging.WARNING)


## Create Kernel and Documents

In [29]:
def create_kernel():
    kernel = Kernel()
    kernel.add_service(
        AzureChatCompletion(
            deployment_name=AZURE_OPENAI_DEPLOYMENT,
            endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            api_version=AZURE_OPENAI_API_VERSION,
        )
    )
    return kernel

def save_sample_documents():
    # Claim Report (Houston scenario)
    claim_report = f"""
INSURANCE CLAIM REPORT
----------------------
Claim ID: HURTX-2024-0456
Date Filed: {datetime.now().strftime('%B %d, %Y')}
Claimant: Contoso Electronics Inc.
Store Location: Contoso Electronics #TX-230
Address: 1125 Main Street, Houston, TX 77002
Contact: Gabriel Allen (Store Manager)
Contact Phone: (713) 555-4422
Contact Email: gabriel.allen@contoso.com

DAMAGE ASSESSMENT
-----------------
Incident Type: Hurricane Alicia
Date of Incident: May 15, 2024
Time of Incident: ~02:30 AM CST

DAMAGE DETAILS
--------------
- Severe roof damage and water intrusion on the south side
- Flooded inventory storage (electronics submerged)
- Electrical panels shorted due to water ingress
- Potential structural compromise in south wing support beams

Preliminary Damage Estimates:
- Inventory Loss: $380,000
- Structural Damage: $200,000
- Equipment Replacement: $160,000
- Business Interruption: TBD (further accounting needed)

Mitigation Steps Taken:
1. Secure site and prevent unauthorized entry
2. Engage emergency restoration team for water extraction
3. Move salvageable inventory to upper floors
4. Temporary roof patch installed

Photos and videos available. Awaiting professional structural engineer’s report.

FILED BY
--------
Adjuster: Lauren Chen
Fabrikam Insurance Adjuster ID: TX-ADJ-3112
Agency: Gulf Coast Claims Solutions
Initial On-Site Visit: {(datetime.now() + timedelta(days=1)).strftime('%B %d, %Y')}
""".strip()

    # Policy Details (Fabrikam Insurance)
    policy_details = f"""
FABRIKAM INSURANCE COMMERCIAL POLICY
-------------------------------------
Policy Number: CP-TX-2024087
Policyholder: Contoso Electronics Inc.
Policy Period: January 1, 2024 - December 31, 2024
Policy Type: Comprehensive Commercial Property Insurance

COVERAGE & LIMITS
-----------------
Total Insured Value: $2,000,000
Hurricane/Wind Coverage: Included
Flood Coverage: Included (Deductible: $50,000)
Business Interruption: Included (sub-limit $500,000)
Replacement Cost: Included

Sublimits:
- Building Structure: Up to $1,000,000
- Business Property (Incl. Inventory): Up to $600,000
- Electronic Equipment: Up to $200,000

DEDUCTIBLES
-----------
Hurricane/Wind Deductible: 5% of total insured value ($100,000)
Flood Deductible: $50,000
Standard Deductible: $25,000

ENDORSEMENTS & NOTES
--------------------
- Hurricane Preparedness Credit applied
- Extended Replacement Cost Endorsement active
- Business Interruption Rider active

UNDERWRITING
------------
Underwriter: Fabrikam Insurance Underwriting Dept.
Last Risk Assessment: April 5, 2024 (No major hazards noted at that time)
Annual Premium: $42,000 paid in full on January 5, 2024
    """.strip()

    # Weather Report (Hurricane Alicia in Houston)
    weather_report = f"""
NOAA SEVERE WEATHER REPORT
--------------------------
Report Date: {datetime.now().strftime('%B %d, %Y')}
Event: Hurricane Alicia (2024 Season)

STORM DETAILS
-------------
Category: 3
Max Sustained Winds: ~120 mph
Min Central Pressure: ~950 mb

HOUSTON AREA IMPACT
-------------------
Landfall: Near Galveston, TX
Time: May 15, 2024, ~02:10 AM CST
Duration of Severe Conditions: 6-8 hours over Houston

METEOROLOGY
-----------
Winds: Sustained 100-120 mph, Gusts 130+ mph
Rainfall: 10-14 inches total, peak 3 in/hr
Storm Surge: 8-10 feet along coastal areas

DAMAGE POTENTIAL
----------------
Infrastructure Risk: Very High
Property Damage Likelihood: High
Business Disruption Probability: Significant

Issued By: Dr. Marisol Reyes
National Hurricane Center - Houston Office
""".strip()

    base_directory = Path.cwd() / "insurance_docs"
    base_directory.mkdir(exist_ok=True)

    documents = {
        "claim_report.txt": claim_report,
        "policy_details.txt": policy_details,
        "weather_report.txt": weather_report,
    }

    for fname, content in documents.items():
        (base_directory / fname).write_text(content, encoding="utf-8")

    return [str(base_directory / f) for f in documents.keys()]

document_files = save_sample_documents()


## Creating Agents

We will have three agents:

1. **Claims Analyst (with File Search)**: Reviews claim, policy, and weather documents.
2. **Policy Verifier**: Checks coverage and compliance.
3. **Risk Assessor**: Evaluates risks and provides recommendations.

In [30]:
async def create_agents(kernel: Kernel, document_files: list[str]):
    claims_analyst = await AzureAssistantAgent.create(
        kernel=kernel,
        deployment_name=AZURE_OPENAI_DEPLOYMENT,
        endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY,
        name="Claims_Analyst",
        instructions="""You are an experienced Insurance Claims Analyst. Your responsibilities:
- On your first turn, only provide initial analysis and identify missing info. DO NOT give a final recommendation yet.
- Wait for the Policy_Verifier and Risk_Assessor to respond before your final recommendation. Your responsibilities:
- Extract and validate claim details from the claim report
- Calculate preliminary damage estimates vs. policy limits
- Identify potential coverage issues
- Flag any missing documentation or inconsistencies

Key focus areas:
- Verify damage aligns with weather event timing and characteristics
- Calculate total claimed amount vs. policy limits
- Identify any immediate red flags or missing information
- Assess adequacy of mitigation steps taken

Available documents:
- Claim Report
- Policy Details
- Weather Report

Provide analysis in JSON-like format:
{
    "findings": [...],
    "concerns": [...],
    "recommendations": [...],
    "confidence_level": "..."
}

If missing/inconsistent info:
1. State what is missing
2. Explain importance
3. Suggest how to obtain it
4. State if partial info is enough

"Do not provide a final recommendation until after the Policy_Verifier and Risk_Assessor have responded. In your first response, only summarize and identify missing info, do not finalize the claim."

""",
        enable_file_search=True,
        vector_store_filenames=document_files
    )

    policy_verifier = ChatCompletionAgent(
        service_id=AZURE_OPENAI_DEPLOYMENT,
        kernel=kernel,
        name="Policy_Verifier",
        instructions="""You are an Insurance Policy Verification Specialist. Responsibilities:
- Confirm coverage and limits for damages
- Apply correct deductibles
- Check policy status and endorsements

Key focus:
- Hurricane vs flood deductibles
- Coverage limits per category
- Identify gaps or exclusions

Format JSON-like:
{
    "verification_results": [...],
    "coverage_gaps": [...],
    "deductible_calculations": [...],
    "recommendations": [...]
}

If issues:
1. Cite policy sections
2. Explain impact
3. Suggest resolution""",
    )

    risk_assessor = ChatCompletionAgent(
        service_id=AZURE_OPENAI_DEPLOYMENT,
        kernel=kernel,
        name="Risk_Assessor",
        instructions="""You are an Insurance Risk Assessment Specialist. Responsibilities:
- Compare reported damage to storm data
- Assess mitigation measures
- Evaluate business interruption risk

Key focus:
- Timing match with hurricane conditions
- Effectiveness of emergency steps
- Potential long-term risks

Format JSON-like:
{
    "risk_findings": [...],
    "mitigation_assessment": [...],
    "future_risks": [...],
    "recommendations": [...],
    "confidence_score": "..."
}

If risk factors:
1. Quantify impact
2. Provide evidence
3. Suggest strategies
4. Note time-sensitive concerns"""
    )

    return claims_analyst, policy_verifier, risk_assessor


## Setting Up Group Chat
We’ll define a selection strategy to determine which agent responds next and a termination strategy to know when the conversation is complete. For demo purposes, we might simplify the logic or leave it as-is.

In [34]:
async def setup_group_chat(
    kernel: Kernel, claims_analyst, policy_verifier, risk_assessor
):
    # Selection function: decides who responds next
    selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt="""
We have three agents: Claims_Analyst, Policy_Verifier, and Risk_Assessor.
They must speak in this order for the claim scenario:
1. Claims_Analyst (initial review only, no final recommendation)
2. Policy_Verifier
3. Risk_Assessor
4. Claims_Analyst (final recommendation after seeing the other two)

Rules:
- If we have just started (user just spoke), choose Claims_Analyst.
- After Claims_Analyst's initial review, choose Policy_Verifier.
- After Policy_Verifier responds, choose Risk_Assessor.
- After Risk_Assessor responds, choose Claims_Analyst for the final recommendation.
- If all these steps are done, the conversation is complete. However, always return the next logical agent if asked.
- Under no circumstances produce any text other than one of the agent names.

Review the conversation history. Based on what has happened so far, output only the single agent name who should speak next.

History:
{{{$history}}}
"""
)


    # Termination function: checks if done
    termination_function = KernelFunctionFromPrompt(
        function_name="termination",
        prompt="""
Finish only when the following steps have each occurred in a separate turn:

1. Claims_Analyst has provided an initial review (no final recommendation yet).
2. Policy_Verifier has verified coverage and provided their analysis.
3. Risk_Assessor has evaluated the risk and provided recommendations.
4. Finally, Claims_Analyst has provided a concluding recommendation after seeing both other agents' inputs.

If these four steps have NOT all clearly occurred, respond with 'INCOMPLETE'.
If all four steps have occurred in the correct order, respond 'COMPLETE'.

History:
{{{$history}}}
""",
    )

    group_chat = AgentGroupChat(
        agents=[claims_analyst, policy_verifier, risk_assessor],
        selection_strategy=KernelFunctionSelectionStrategy(
            function=selection_function,
            kernel=kernel,
            result_parser=lambda r: str(r.value[0]) if r.value else "Claims_Analyst",
            agent_variable_name="agents",
            history_variable_name="history",
        ),
        termination_strategy=KernelFunctionTerminationStrategy(
            agents=[claims_analyst],
            function=termination_function,
            kernel=kernel,
            result_parser=lambda r: "complete" in str(r.value[0]).lower(),
            history_variable_name="history",
            maximum_iterations=20,
        ),
    )

    return group_chat

## Processing a Claim
This function executes the group chat workflow, starting with a user message (the claim scenario). We print out agent responses and track progress. In a real scenario, you’d refine the logic further or handle streaming differently.

In [35]:
async def process_claim(group_chat: AgentGroupChat, claim_id: str):
    print(f"🔄 Starting analysis for claim {claim_id}")
    await group_chat.add_chat_message(
        ChatMessageContent(role=AuthorRole.USER, content=f"Please analyze claim {claim_id} due to Hurricane Alicia in Houston and provide next steps.")
    )

    current_agent = None
    chunks = []

    async for response in group_chat.invoke_stream():
        # When we detect a new agent turn, print the previous agent's fully assembled response
        if response.name != current_agent:
            if current_agent and chunks:
                # Join chunks without extra spaces, then normalize whitespace
                final_text = ''.join(chunks)
                final_text = ' '.join(final_text.split())
                print(f"\n👤 {current_agent}:\n{final_text}")
            current_agent = response.name
            chunks = []

        if response.content:
            # Strip and store chunks, no extra spaces
            chunks.append(response.content.strip())

    # Print any remaining response after the loop
    if current_agent and chunks:
        final_text = ''.join(chunks)
        final_text = ' '.join(final_text.split())
        print(f"\n👤 {current_agent}:\n{final_text}")

    print("\n✅ Claim analysis complete!")


## Run the Entire Workflow

In this final cell, we:

- Initialize kernel & documents
- Create agents
- Setup group chat
- Process the claim

Run this cell and review the outputs. You can add additional Markdown cells afterwards to comment on the results.

In [36]:
kernel = create_kernel()
claims_analyst, policy_verifier, risk_assessor = asyncio.run(create_agents(kernel, document_files))
group_chat = asyncio.run(setup_group_chat(kernel, claims_analyst, policy_verifier, risk_assessor))

asyncio.run(process_claim(group_chat, "HURTX-2024-0456"))

# Cleanup
asyncio.run(claims_analyst.delete())


🔄 Starting analysis for claim HURTX-2024-0456

👤 Claims_Analyst:
{  " find ings ": [  {  " claim _id ": " H UR TX - 202 4 - 045 6 ",  " claim ant ": " Cont oso Electronics Inc .",  " incident _type ": " H urricane Alicia ",  " date _of _inc ident ": " May  15 ,  202 4 ",  " damage _details ": {  " se vere _ro of _damage ": true ,  " water _intr usion ": true ,  " f lood ed _inventory ": true ,  " elect rical _pan els ": true ,  " struct ural _comp romise ": true   },  " pre liminary _damage _est imates ": {  " inventory _loss ":  380 000 ,  " struct ural _damage ":  200 000 ,  " equipment _re placement ":  160 000 ,  " business _inter ruption ": " T BD "  },  " mit igation _steps _taken ": [  " Secure site and prevent unauthorized entry ",  " Eng age emergency restoration team for water extraction ",  " Move salvage able inventory to upper floors ",  " Temporary roof patch installed "  ],  " weather _ impact ": {  " category ":  3 ,  " max _s ust ained _w inds ":  120 ,  " rain fall ":

True